In [1]:
from __future__ import annotations

from pathlib import Path
WORK_DIR = Path.cwd()

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import smtplib

## Testing Alpha Vantage API

In [2]:
from alpha_vantage.timeseries import TimeSeries

ts = TimeSeries(key='UX2OT39HH6HK14LB')
data, meta_data = ts.get_intraday('AAPL')

In [3]:
import numpy as np
import pandas as pd
import requests
import time

class AlphaVantageDataLoader():
    def __init__(
                 self, 
                 key: str):
        
        self.key=key
        self.tickers = pickle.load(open('updated_TSX_ticker.pkl', 'rb'))

    
    def load_stock_data(self,
        tickers: list = [],
        frequency: str = 'daily', 
        output_size: str = 'full'
        ) -> dict: 

        freq_dict = {'intraday': 'TIME_SERIES_INTRADAY',
                     'daily': 'TIME_SERIES_DAILY',
                     'weekly': 'TIME_SERIES_WEEKLY',
                     'monthly': 'TIME_SERIES_MONTHLY'}
        
        if type(tickers) == str:
            tickers = [tickers]

        start = time.time()
        meta_dict = {} 
        df_dict = {}
        count=0
        
        for num, ticker in tqdm(enumerate(tickers)):
            link = f'https://www.alphavantage.co/query?function={freq_dict[frequency]}&symbol={ticker}&outputsize={output_size}&apikey={self.key}'
            stock = requests.get(link)
            keys_list = list(stock.json().keys())
            
            try:
                data, metadata = stock.json()[keys_list[-1]], stock.json()[keys_list[0]]

                df = pd.DataFrame(data).T
                df.columns = [col.split(' ')[-1] for col in df.columns]
                df = df.apply(pd.to_numeric)

                meta_dict[ticker] = meta_data
                df_dict[ticker]= df
                count+=1
            except:
                pass

            #Alpha Vantage only allows 5 API call per minute
            if (count+1)%5==0: 
                end = time.time()
                wait_time = (start+60) - end
                time.sleep(wait_time)
                start = time.time()
        
        self.meta = meta_dict
        self.loaded_data = df_dict
        return df_dict
    


In [51]:
import random
import pickle

ticker = pd.read_csv(WORK_DIR/'TSX.txt', sep='\t')
ticker_list = ticker.loc[(~ticker.Symbol.str.contains('PR'))&
                           (~ticker.Description.str.contains('ETF'))&
                           (~ticker.Description.str.contains('Fund'))&
                           (~ticker.Description.str.contains('hdg'))]['Symbol'].tolist()

name_reference = ticker.set_index('Symbol')['Description'].T.to_dict()
sampled_ticker = random.sample(ticker_list, 10)

update_dict = {'TECK-B.TO': 'Teck Resources'}
name_reference.update(update_dict)
pickle.dump(name_reference, open('updated_TSX_ticker.pkl', 'wb'))

In [6]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
#stock_data = loader.load_stock_data(tickers=sampled_ticker)

### Value Threshold Announcement`

In [64]:
def generate_report(stock_data, higher_threshold=None, lower_threshold=None):
    stock_report = [] 
    lower_threshold_report = [] 
    higher_threshold_report = []
    
    name_reference = pickle.load(open('updated_TSX_ticker.pkl', 'rb'))
    
    for ticker in stock_data.keys():
        df1 = stock_data[ticker].reset_index()
        df1['index'] = pd.to_datetime(df1['index'])
        df1.set_index('index', inplace=True)

        weekly_close = df1.resample('W')['close'].mean().tail(52)
        current_close = weekly_close.iloc[-1]

        arr = (weekly_close>weekly_close.shift(1)).astype(int).iloc[::-1].values

        if np.any(arr==0):
            consecutive_week_high = np.where(arr==0)[0][0]
        else:
            consecutive_week_high = np.where(arr==0)[0][0]

        date = stock_data[ticker].index[0]
        delta = (stock_data[ticker].iloc[0]['close'] - stock_data[ticker].iloc[1]['close'])/stock_data[ticker].iloc[1]['close']*100
        close_price = stock_data[ticker].iloc[0]['close']
        ma50 = stock_data[ticker].iloc[0:50]['close'].mean()
        ma50_pct = (close_price - ma50)/ma50  * 100

        ma200 = stock_data[ticker].iloc[0:200]['close'].mean()
        ma200_pct = (close_price - ma200)/ma200 * 100

        try: 
            ticker_name = name_reference[ticker]
        except:
            ticker_name = 'Name Missing'

        str1 = f"{ticker_name} ({ticker}): {date}\nClose Price: {close_price}\n% Changes: {delta:.2f}%\nMA50: {ma50:.2f} ({ma50_pct:.2f}%)\nMA200: {ma200:.2f} ({ma200_pct:.2f}%)\nConsecutive Weeks High: {consecutive_week_high}"

        #Sorting report by stock with highest consecutive week high to get a pulse on good growth stock
        stock_report.append((consecutive_week_high, str1))

    sorted_list = sorted(stock_report, key=lambda x: x[0], reverse=True)
    stock_report = [x[1] for x in sorted_list]
    
    ################## Checking Threshold #################
    if higher_threshold is not None:
        for ticker, value in higher_threshold.items():
            if ticker in stock_data.keys():
                close_price = stock_data[ticker].iloc[0]['close']
                if close_price>value:
                    str1 = f"{name_reference[ticker]} ({ticker}): Closing Price {close_price:.2f}CAD is above threshold of {value:.2f}CAD"
                    higher_threshold_report.append(str1)
    if lower_threshold is not None:
        for ticker, value in lower_threshold.items():
            if ticker in stock_data.keys():
                close_price = stock_data[ticker].iloc[0]['close']
                if close_price<value:
                    str1 = f"{name_reference[ticker]} ({ticker}): Closing Price {close_price:.2f}CAD is below threshold of {value:.2f}CAD"
                    lower_threshold_report.append(str1)
    
    ################## Finding Missing Stock Tickers #############################
    missing_list = [ticker for ticker in download_list if ticker not in stock_data.keys()]
    
    ################## Combing all sections into a single report #################
    output_list = [] 

    joined_report = "\n\n".join(stock_report)

    output_list.append(joined_report)
    if len(higher_threshold_report)>0:
        output_list.append("\n".join(higher_threshold_report))

    if len(lower_threshold_report)>0:
        output_list.append("\n".join(lower_threshold_report))

    output_list.append(f'{", ".join(missing_list)} are missing')
    output = '\n\n#################################################################\n\n'.join(output_list)
    
    return output

In [77]:
sampled_ticker = random.sample(ticker_list, 40)


In [78]:
higher_threshold = {'BB.TO': 20, 'CGX.TO': 16.5, 'BDLP.TO': 20}
lower_threshold = {'BB.TO': 10, 'CGX.TO': 15, 'BDLP.TO': 30}
report_ticker = ['BB.TO', 'CGX.TO', 'L.TO', 'BDLP.TO', 'TECK-B.TO'] + sampled_ticker

download_list = list(set(list(higher_threshold.keys()) + list(lower_threshold.keys()) + report_ticker))

In [80]:
# Initially wanted to use intraday data but intraday from alpha vantage doesn't seem to work for canadian stocks
loader = AlphaVantageDataLoader(key='UX2OT39HH6HK14LB')
stock_data = loader.load_stock_data(tickers=download_list)

45it [01:24,  1.89s/it]


In [81]:
output = generate_report(stock_data, higher_threshold = higher_threshold, lower_threshold = lower_threshold)

In [82]:
print(output)

Blackberry Limited (BB.TO): 2021-06-11
Close Price: 17.21
% Changes: 2.38%
MA50: 12.07 (42.54%)
MA200: 10.65 (61.63%)
Consecutive Weeks High: 4

Talon Metals Corp (TLO.TO): 2021-06-11
Close Price: 0.64
% Changes: 1.59%
MA50: 0.64 (0.09%)
MA200: 0.50 (27.61%)
Consecutive Weeks High: 3

Algonquin Power and Utilities Corp (AQN.TO): 2021-06-11
Close Price: 19.46
% Changes: 0.99%
MA50: 19.44 (0.08%)
MA200: 20.12 (-3.29%)
Consecutive Weeks High: 3

Sustainable Power Infra Split Corp Cl A (PWI.TO): 2021-06-11
Close Price: 9.55
% Changes: 0.53%
MA50: 9.45 (1.10%)
MA200: 9.45 (1.10%)
Consecutive Weeks High: 1

Apollo Healthcare Corp. (AHC.TO): 2021-06-11
Close Price: 3.17
% Changes: -2.46%
MA50: 3.66 (-13.33%)
MA200: 4.08 (-22.21%)
Consecutive Weeks High: 1

Purp Gold Bullion Tu (KILO.TO): 2021-06-11
Close Price: 28.49
% Changes: -1.04%
MA50: 27.65 (3.05%)
MA200: 28.29 (0.71%)
Consecutive Weeks High: 0

Aimia Inc (AIM.TO): 2021-06-11
Close Price: 4.77
% Changes: -0.21%
MA50: 4.92 (-3.01%)
MA200

#### Sending Notification Email

In [83]:
sender = "tuateststock@outlook.com" 
receiver = "wongsangaroon@yahoo.co.th"
password = ("ImTestingStocks123") 
message = output

In [84]:
server = smtplib.SMTP('smtp-mail.outlook.com', 587)
server.starttls()
server.login(sender, password)
print("Login Success")
server.sendmail(sender, receiver, msg = message) 
print("Email was sent")
server.close()

Login Success
Email was sent
